Ref: https://github.com/NCAR/cesm-lens-aws/issues/34

In [ ]:
import xarray as xr
import intake
from tqdm.auto import tqdm
import shutil 
import os
from functools import reduce
import pprint
import json
from operator import mul
import random
import yaml

import numpy as np
import pandas as pd

import cftime
from datetime import date

## Calendar Conversion functions

In [ ]:
# Functions for converting single date objects from one type to another.

def convert_to_noleap(cftime360_obj, datemap):
    ''' Convert Date from 360 Day to NoLeap'''
    newdate = datemap[cftime360_obj.dayofyr - 1]
    converted = cftime.DatetimeNoLeap(year=cftime360_obj.year, month=newdate.month, day=newdate.day)
    return converted

def convert_to_gregorian(cftime_noleap_obj):
    ''' Convert Date from NoLeap to Gregorian '''
    converted = cftime.DatetimeGregorian(year=cftime_noleap_obj.year, month=cftime_noleap_obj.month, day=cftime_noleap_obj.day)
    return converted

def convert_hour(time_obj, hour_of_day):
    ''' Convert date object to Gregorian and explicitly set the hour of day.'''
    time_obj = cftime.DatetimeGregorian(year=time_obj.year, month=time_obj.month, day=time_obj.day, hour=hour_of_day, minute=0, second=0)
    return time_obj

In [ ]:
def get_datemap_360_to_noleap():
    ''' Return an array of dates mapping days from the 360-Day calendar to the No-Leap calendar. '''

    # Choose any year with 365 days. 
    dummy_year = 1999

    # These are the days of the year that will be missing on the time axis for each year.
    # The goal is to spread missing dates out evenly over each year.
    #
    # Modify specific dates as desired. 
    missing_dates = [date(dummy_year, 1, 31),
                     date(dummy_year, 3, 31),
                     date(dummy_year, 5, 31),
                     date(dummy_year, 8, 31),
                     date(dummy_year, 10, 31),]
    
    day_one = date(dummy_year, 1, 1)
    missing_dates_indexes = [(day - day_one).days + 1 for day in missing_dates] 
    missing_dates_indexes

    datemap_indexes = np.setdiff1d(np.arange(365), missing_dates_indexes)
    datemap_indexes

    dates = pd.date_range(f'1/1/{dummy_year}', f'12/31/{dummy_year}')
    assert(len(dates) == 365)
    
    date_map = dates[datemap_indexes]
    assert(len(date_map) == 360)
    
    # Check to make sure February 29 is not a date in the resulting map.
    #is_leap_day = [(d.month == 2) and (d.day == 29) for d in date_map]
    #print(is_leap_day)
    #assert(not any(is_leap_day))
    return date_map


# Create a global map for moving days of the year to other days of the year.
datemap_global = get_datemap_360_to_noleap()

### Calendar Padding Code Used for Cases without Standard Calendars Included

In [ ]:
def convert_dataset_noleap_to_gregorian(ds):
    '''Converts an xarray dataset from the NoLeap calendar to the Gregorian calendar.  
       Data for Leap Days are filled with missing values (np.nan).
    '''
    # Convert dates in the original dataset from the NoLeap to Gregorian calendar
    ds['time'] = [convert_to_gregorian(t) for t in ds.time.values]
    
    # Create an equivalent date range on the Gregorian calendar
    start_date = ds.time.values[0]
    end_date = ds.time.values[-1]
    times = xr.DataArray(xr.cftime_range(start=start_date, end=end_date, freq='D', calendar='gregorian', normalize=True), dims='time')
    
    # Find the leap days in this date range.
    is_leap_day = (times.time.dt.month == 2) & (times.time.dt.day == 29)
    leap_days = times.where(is_leap_day, drop=True)
    
    # Create fill values for these days.
    one_time_step = ds.isel(time=slice(0, 1))
    fill_values = []
    for leap_day in leap_days:
        d = xr.full_like(one_time_step,fill_value=np.nan)
        d = d.assign_coords(time=[leap_day.data])
        fill_values.append(d)
    
    ## EXPERIMENTAL SECTION
    # Append the fill values to the dataset and then sort values by time.
    fill_values.append(ds)
    
    ds_fixed=xr.concat(fill_values, dim='time').sortby('time')
    #ds_fixed=xr.merge([ds, fill_values]).sortby('time')

    #ds_fixed = ds_fixed.assign_coords(time=times)

    return ds_fixed 

## Run These Cells for Dask Processing

In [ ]:
import dask
from ncar_jobqueue import NCARCluster

# Processes is processes PER CORE.
# This one works fine.
#cluster = NCARCluster(cores=15, processes=1, memory='100GB', project='STDD0003')
# This one also works, but occasionally hangs near the end.
#cluster = NCARCluster(cores=10, processes=1, memory='50GB', project='STDD0003')

num_jobs = 5
walltime = "4:00:00"
cluster = NCARCluster(cores=num_jobs, processes=1, memory='100GB', project='STDD0003', walltime=walltime)
cluster.scale(jobs=num_jobs)

from distributed import Client
from distributed.utils import format_bytes
client = Client(cluster)
cluster

In [ ]:
cluster.close()

In [ ]:
cluster

In [ ]:
cluster.adapt(minimum_jobs=4, maximum_jobs=8)
#cluster.scale(jobs=14)
client = Client(cluster)
cluster

In [ ]:
# Set to True if saving large Zarr files is resulting in KilledWorker or Dask crashes.
BIG_SAVE = False
if BIG_SAVE:
    min_workers = min_jobs
    print('Waiting for ' + str(min_jobs) + ' workers.')
    client.wait_for_workers(min_workers)

## Main Notebook Code

#### Prepare individual dataset for merge

In [ ]:
def preprocess(ds):
    """This function gets called on each original dataset before concatenation.
       Convert all dataset calendars to Gregorian.  
       For now, also drop other data variables, like time bounds, until we get things looking good.
    """

    # Drop any time bounds data, as this will get regenerated at a later step.
    if 'time_bnds' in ds.data_vars:
        ds_fixed = ds.drop('time_bnds')
    else:
        ds_fixed = ds
    
    time_values = ds.time.values
    
    attrs = ds.time.attrs
    encoding = ds.time.encoding

    #"""Drop all unneeded variables and coordinates"""
    #vars_to_drop = [vname for vname in ds.data_vars if vname not in variables]
    #coord_vars = [vname for vname in ds.data_vars if 'time' not in ds[vname].dims or 'bnd' in vname]
    #ds_fixed = ds.set_coords(coord_vars)
    #data_vars_dims = []
    #for data_var in ds_fixed.data_vars:
    #    data_vars_dims.extend(list(ds_fixed[data_var].dims))
    #coords_to_drop = [coord for coord in ds_fixed.coords if coord not in data_vars_dims]
    #grid_vars = list(set(vars_to_drop + coords_to_drop) - set(['time', 'time_bound']))
    #ds_fixed = ds_fixed.drop(grid_vars)
    #if 'history' in ds_fixed.attrs:
    #    del ds_fixed.attrs['history']
    
    # Test for calendar type xarray found when it loaded the dataset.
    time_type = f'{type(time_values[0])}'
    has_360_day_calendar = "Datetime360Day" in time_type
    has_noleap_calendar = "DatetimeNoLeap" in time_type
    
    # Extract the time_bnds variable for conversion
    #bnds = ds_fixed[bounds_name].values

    if has_360_day_calendar:
        print(f'Found 360 day calendar.\n')
        time_values = [convert_to_noleap(t, datemap_global) for t in time_values]
        time_values = [convert_to_gregorian(t) for t in time_values]

    # Convert any NoLeap calendar to the Gregorian calendar.
    elif has_noleap_calendar:
        print(f'Found NoLeap calendar.\n')
        time_values = [convert_to_gregorian(t) for t in time_values]
        #ds_fixed = convert_dataset_noleap_to_gregorian(ds_fixed)

    # Change time of day to noon for all time axis points.
    ###print(ds_fixed.time.values.shape)
    time_values = [convert_hour(t, 12) for t in time_values]
    
    ds_fixed = ds_fixed.assign_coords(time = time_values)
    ds_fixed.time.attrs = attrs
        
    #ds = ds.set_coords([bounds_name])

    return ds_fixed

#### Merged dataset processing functions

In [ ]:
def fix_time(
    ds,
    start,
    end,
    freq,
    time_bounds_dim,
    calendar='standard',
    generate_bounds=True,
    instantaneous=False,
):
    '''Regenerate time axis to be consistent with time bounds variable'''
    ds = ds.sortby('time').copy()
    attrs = ds.time.attrs
    encoding = ds.time.encoding
    
    # The bounds name should always be the same.
    bounds_name = 'time_bnds'

    if generate_bounds:
        times = xr.cftime_range(
            start=start, end=end, freq=freq, calendar=calendar
        )
        bounds = np.vstack([times[:-1], times[1:]]).T
        ds = ds.assign_coords({bounds_name: (("time", "bnds"), bounds)})
        
    if instantaneous:
        ds = ds.assign_coords(time=ds[bounds_name].min(time_bounds_dim))
    else:
        ds = ds.assign_coords(time=ds[bounds_name].mean(time_bounds_dim))

    ds.time.attrs = attrs
    ds.time.encoding = encoding


    return ds

In [ ]:
def enforce_chunking(datasets, chunks, data_var):
    """Enforce uniform chunking in the Zarr Store.
    """
    dsets = datasets.copy()
    choice = random.choice(range(0, len(dsets)))
    for i, (key, ds) in enumerate(dsets.items()):
        INSERT_LEAP_DAYS = False
        if INSERT_LEAP_DAYS:
            ds = convert_dataset_noleap_to_gregorian(ds)
        print(f'key == {key}')
        c = chunks.copy()
        for dim in list(c):
            if dim not in ds.dims:
                del c[dim]
        ds = ds.chunk(c)
        keys_to_delete = ['intake_esm_dataset_key', 'intake_esm_varname']
        for k in keys_to_delete:
            del ds.attrs[k]
        dsets[key] = ds
        #variable = key.split(field_separator)[-1]
        #print_ds_info(ds, variable)
        print_ds_info(ds, data_var)
        if i == choice:
            print(ds)
        print('\n')
    return dsets

In [ ]:
def print_ds_info(ds, var):
    """Function for printing chunking information"""

    print(f'print_ds_info: var == {var}')
    dt = ds[var].dtype
    itemsize = dt.itemsize
    chunk_size = ds[var].data.chunksize
    size = format_bytes(ds.nbytes)
    _bytes = reduce(mul, chunk_size) * itemsize
    chunk_size_bytes = format_bytes(_bytes)

    print(f'Variable name: {var}')
    print(f'Dataset dimensions: {ds[var].dims}')
    print(f'Chunk shape: {chunk_size}')
    print(f'Dataset shape: {ds[var].shape}')
    print(f'Chunk size: {chunk_size_bytes}')
    print(f'Dataset size: {size}')

# For now, make the Zarr output directory a global variable.
dirout = '/glade/scratch/bonnland/na-cordex/zarr'

def zarr_store(var, exp, frequency, grid, biascorrection, write=False, dirout=dirout):
    """ Create zarr store name/path
    """
    path = f'{dirout}/{var}.{exp}.{frequency}.{grid}.{biascorrection}.zarr'
    if write and os.path.exists(path):
        shutil.rmtree(path)
    print(path)
    return path


def save_data(ds, store):
    try:
        ds.to_zarr(store, consolidated=True)
        del ds
    except Exception as e:
        print(f"Failed to write {store}: {e}")

        
def zarr_check():
    '''Make sure the zarr stores were properly written'''

    from pathlib import Path
    p = Path(dirout)
    #stores = list(p.rglob("*.zarr"))
    stores = list(p.rglob("*.rcp45.day.NAM-22i.raw.zarr"))
    for store in stores:
        try:
            ds = xr.open_zarr(store.as_posix(), consolidated=True)
            print('\n')
            print(store)
            print(ds)
        except Exception as e:
            #print(e)
            print(store)

#### Metadata preparation functions

In [ ]:
def get_dataset_metadata(ds, member_id):
    '''Convert all dataset metadata to dictionary form and return for filtering.
    '''
    m_dict = {}
    for key, value in ds.attrs.items():
        m_dict[key] = value
    return m_dict


def update_dict(dict_in, key, value):
    '''Create or append key-value pair to dictionary and return updated dict'''
    if key not in dict_in:
        dict_in[key] = [value]
    else:
        dict_in[key].append(value)
    return dict_in


def is_uniform(metadata, field):
    '''Determines if the given field has uniform values across the raw metadata'''
    member_ids = metadata.keys()
    values = [metadata[member][field] for member in member_ids if field in metadata[member]]
    fileNames = [metadata[member]['fileName'] for member in member_ids if field in metadata[member]]
    is_uniform = all(elem == values[0] for elem in values)
    return is_uniform, values, fileNames

    
def collect_raw_metadata(catalog_entries, target_metadata):
    '''Take a catalog subset and extract target metadata into a dictionary of dictionaries.
    '''    
    metadata_raw = {}

    # Loop over catalog rows
    dataframe = catalog_entries.df
    for path, member_id in zip(dataframe['path'], dataframe['member_id']):
        #ds = xr.open_dataset(path, decode_cf=False, engine="h5netcdf")
        ds = xr.open_dataset(path, decode_cf=False)
        metadata_raw[member_id] = ds.attrs
        metadata_raw[member_id]['fileName'] = path.split('/')[-1]
    
    return metadata_raw

    
def get_metadata_from_catalog_entries(catalog_entries, target_metadata):
    '''Take a catalog subset and extract target metadata to a single dictionary.
    '''
    metadata_raw = collect_raw_metadata(catalog_entries, target_metadata)
    metadata = {}
    
    # Record non-uniform entries among the 'keep_first' fields.
    non_uniform = {}
    for field in target_metadata['keep_first']:
        uniform, values, fileNames = is_uniform(metadata_raw, field)
        if not uniform:
            for value, fileName in zip(values, fileNames):
                dict_entry = (value, fileName)
                non_uniform = update_dict(non_uniform, field, dict_entry)

    # Record missing entries. 
    missing = {}
    for field in target_metadata['keep_first'] + target_metadata['keep_all']:
        for member_id in metadata_raw.keys():
            if field not in metadata_raw[member_id]:
                missing = update_dict(missing, field, metadata_raw[member_id]['fileName'])

    # Produce zarr metadata.
    for field in target_metadata['keep_first']:
        for member_id in metadata_raw.keys():
            if (field not in metadata) and field in metadata_raw[member_id]: 
                metadata[field] = metadata_raw[member_id][field]
                
    for field in target_metadata['keep_all']:
        metadata[field] = {}
        for member_id in metadata_raw.keys():
            if field in metadata_raw[member_id]:
                metadata[field][member_id] = metadata_raw[member_id][field]
                
    # If "contact_note" field has no entries, delete the empty dictionary from the metadata.
    if "contact_note" in metadata and not metadata["contact_note"]:
        metadata.pop("contact_note", None)
        
    # Missing contact_note entries are expected, so remove them.
    if "contact_note" in missing:
        missing.pop("contact_note", None)
    
    
    # Serialize non-uniform metadata dictionaries.  
    # To eventually turn them back into dictionaries, use the json.loads() function.
    for field in target_metadata['keep_all']:
        if field in metadata:
            metadata[field] = json.dumps(metadata[field])

    return metadata, missing, non_uniform


def write_metadata_output(store_name, metadata, missing, non_uniform):
    '''Write the metadata to store_name.out, and warnings to store_name.err'''

    # Metadata dictionary fields have to be de-serialized to print nicely.
    deserialized_metadata = {}
    for key, val in metadata.items():
        try: 
            deserialized_metadata[key] = json.loads(val)
        except Exception as e:
            deserialized_metadata[key] = val
            
    out = open(f'./zarr-metadata/{store_name}.out', 'w')
    pprint.pprint(deserialized_metadata, width=150, stream=out, compact=True)
    out.close()
    
    if missing:
        err = open(f'./zarr-metadata/{store_name}.missing.err', 'w')
        for field in missing.keys():
            for file_name in missing[field]:
                err.write(f'{field}\t{file_name}\n')
        err.close

    if non_uniform:
        err = open(f'./zarr-metadata/{store_name}.inconsistent.err', 'w')
        for field in non_uniform.keys():
            for (value, file_name) in non_uniform[field]:
                err.write(f'{field}\t"{value}"\t{file_name}\n')
        err.close()
            

def save_metadata_to_csv(metadata_dict, variable_name):
    '''Save metadata in dictionary form to a csv file. '''
    dataframe = pd.DataFrame.from_dict(metadata_dict)
    dataframe.to_csv(f'{variable_name}.csv')

## Batch Processing Code Using the Configuration File "config.yaml"

In [ ]:
# It's safer to use a underscore separator, because NA-CORDEX grids have dashes.
field_separator = '_'
col = intake.open_esm_datastore("./glade-na-cordex-bonnland.json", sep=field_separator)
#col = intake.open_esm_datastore("./bogus-na-cordex.json", sep=field_separator)
col

In [ ]:
def process_variables(col, variable, scenario, frequency, grid, biascorrection, verbose=True):
    '''Returns a Zarr Store Creation Spec.'''
    query = dict(variable=variable, scenario=scenario, frequency=frequency, grid=grid, biascorrection=biascorrection)
    subset = col.search(**query)
    if verbose:
        print(subset.unique(columns=['variable', 'scenario', 'frequency', 'grid', 'biascorrection']))
    return subset, query

In [ ]:
# Read the Configuration File.
with open("config.yaml") as f:
    config = yaml.safe_load(f)
        
variables = config['variables']
frequencies = config['frequencies']
scenarios = config['scenarios']
biascorrections = config['biascorrections']
grid_categories = config['grid_categories']

target_metadata = config['target_metadata']
target_metadata

In [ ]:
# Produce a list of Zarr Store Creation Specs.
run_config = []
for key, value in grid_categories.items():
    grid = value['grid']
    chunks = value['chunks']
    for scenario in scenarios:
        for frequency in frequencies:
            for biascorrection in biascorrections:
                for variable in variables:
                    col_subset, query = process_variables(col, variable, scenario, frequency, grid, biascorrection, verbose=False)
                    d = {'query': json.dumps(query), 'col': col_subset, 'chunks': chunks, 'frequency': frequency, 'variable': variable}
                    run_config.append(d)
                    
run_config

In [ ]:
# Produce Zarr Stores as specified in config.yaml.
for run in run_config:
    print("*"*120)
    print(f"query = {run['query']}")
    frequency = run['frequency']
    chunks = run['chunks'].copy()
    var = run['variable']
    
    qry = json.loads(run['query'])
    exp = qry['scenario']
    grid = qry['grid']
    biascorrection = qry['biascorrection']
    
    # Skip cases where Zarr stores exist already.
    zarr_path = f'{dirout}/{var}.{exp}.{frequency}.{grid}.{biascorrection}.zarr'
    if os.path.exists(zarr_path):
        print(f'Store exists, skipping: {zarr_path}')
        continue

    # Gather target Zarr metadata from catalog subset.
    store_metadata, missing, non_uniform = get_metadata_from_catalog_entries(run['col'], target_metadata)
    #pprint.pprint(store_metadata)
    
    CHECK_METADATA = False
    if CHECK_METADATA:
        # Loop over input files and print various metadata fields.
        for f in run['col'].df['path']:
            print(f)
            tf = xr.open_dataset(f)
            print(tf.time.attrs)
            print(tf.data_vars)
            print(tf.coords)
            print('############\n')
    
    # Try preprocessing, including calendar conversion.
    dsets = run['col'].to_dataset_dict(cdf_kwargs={'chunks': chunks, 'use_cftime': True}, preprocess=preprocess, progressbar=False)

    final_chunks = chunks.copy()
    final_chunks['member_id'] = 4

    dsets = enforce_chunking(dsets, final_chunks, var)

    for key, ds in tqdm(dsets.items(), desc='Creating zarr store if not present'):
        print('key: ' + key)
        key = key.split(field_separator)
        exp, frequency, grid, biascorrection = key[0], key[1], key[2], key[3]


        # Regenerate the time bounds variable to be consistent across all ensemble members.
        #
        # start:  Move the starting bound backward from noon to midnight of the first day.
        # end:    Create an extra day for the ending time bound of the last day, and set hour to midnight.
        start = convert_hour((ds.time.values[0]), 0)
        end = convert_hour(pd.to_datetime(ds.time.values[-1].strftime()) + pd.DateOffset(1), 0)
        time_bounds_dim='bnds'
        ds_fixed = fix_time(ds, start=start, end=end, freq='D', time_bounds_dim=time_bounds_dim).chunk(final_chunks)
        
        # Insert target metadata.
        ds_fixed.attrs = store_metadata
       
        store = zarr_store(var, exp, frequency, grid, biascorrection, write=True, dirout=dirout)
        save_data(ds_fixed, store)

In [ ]:
# Check to see if Zarr Stores were saved properly.
zarr_check()

### If Using Dask on HPC, release the workers.

In [ ]:
!date

In [ ]:
cluster.close()

In [ ]:
# Use this to print out details about the conda environment.
# %load_ext watermark
# %watermark -d -iv -m -g -h

## Create Single Zarr Store, For Troubleshooting

In [ ]:
# It's safer to use a underscore separator, because NA-CORDEX grids have dashes.
field_separator = '_'
col = intake.open_esm_datastore("./glade-na-cordex-bonnland.json", sep=field_separator)
col

In [ ]:
# Subset to a single target Zarr store.
subset = col.search(variable='sfcWind', scenario='hist', frequency='day', grid='NAM-22i', biascorrection='raw')
subset

In [ ]:
# Print information from input files.
for file in subset.df['path']:
    ds = xr.open_dataset(file, use_cftime=True)
    print(file + '\n')
    print(f'[{ds.time.values[0]},  ..., {ds.time.values[-3]},  {ds.time.values[-2]},  {ds.time.values[-1]}]')
    print('\n')

In [ ]:
# Example of isolating one entry from the catalog.
#ds = col['/Users/bonnland/GitRepos/cesm-lens-zarrification/notebooks/na-cordex/data-subsets/subset_tasmax.rcp85.CanESM2.CRCM5-UQAM.day.NAM-22i.raw.nc_tasmax_rcp85_CanESM2_CRCM5-UQAM_day_NAM-22i_raw_common_CanESM2.CRCM5-UQAM'].to_dask()
#dict(ds.dims)

In [ ]:
# Hard-code the variable name in a global variable for now.
variables = ['hurs']

In [ ]:
# Consolidate datasets according to the catalog JSON metadata.
chunks = {'time': 1000, 'lat': 65, 'lon': 120}
dsets = subset.to_dataset_dict(cdf_kwargs={'chunks': chunks, 'use_cftime': True}, preprocess=preprocess, progressbar=True)
#dsets = subset.to_dataset_dict(cdf_kwargs={'chunks': chunks, 'use_cftime': True}, preprocess=preprocess, aggregate=False, progressbar=False)
#dset = dsets['rcp85_day_NAM-22i_raw']
#dset

In [ ]:
dset = dsets['hist_day_NAM-22i_raw']
dset

In [ ]:
# The following line will place all ensemble members in the same chunk.   
# Comment out to have each ensemble member in its own chunk.
chunks = {'member_id': 4, 'lat': 65, 'lon':120, 'time': 1000}
#chunks['member_id'] = 4
#chunks['time'] = 20

# Take care of ragged edges in original datasets, to optimize chunking strategy.
dsets = enforce_chunking(dsets, chunks, variables[0])
dsets['hist_day_NAM-22i_raw']

In [ ]:
# Create/Overwrite the Zarr Stores.
dsets['hist_day_NAM-22i_raw']

In [ ]:
for key, ds in dsets.items():
    print('key: ' + key)
    key = key.split(field_separator)
    exp, frequency, grid, biascorrection = key[0], key[1], key[2], key[3]
    
    # Regenerate the time bounds variable to be consistent across all ensemble members.
    #
    # start:  Move the starting bound backward from noon to midnight of the first day.
    # end:    Create an extra day for the ending time bound of the last day, and set hour to midnight.
    start = convert_hour((dset.time.values[0]), 0)
    end = convert_hour(pd.to_datetime(dset.time.values[-1].strftime()) + pd.DateOffset(1), 0)
    time_bounds_dim='bnds'
    ds_fixed = fix_time(dset, start=start, end=end, freq='D', time_bounds_dim=time_bounds_dim).chunk(chunks)

    var = variables[0]
    store = zarr_store(var, exp, frequency, grid, biascorrection, write=True, dirout=dirout)
    print(store)
    save_data(ds_fixed, store)

In [ ]:
ds_fixed

In [ ]:
names = [name for name in run['col'].keys()]
names[0]

In [ ]:
run_config

## Save Target Metadata and Warnings for Stores in config.yaml

In [ ]:
len(run_config)

In [ ]:
for run in run_config:
    print("*"*120)
    print(f"query = {run['query']}")

    var = run['variable']
    store_names = [name for name in run['col'].keys()]
    
    # Filter out bogus stores with no associated NetCDF data.
    if len(store_names) == 0:
        continue

    assert(len(store_names) == 1)
    store_name = f'{var}_{store_names[0]}'
    store_name = store_name.replace('_', '.')
    
    # Gather target Zarr metadata from catalog subset.
    SKIP_PROBLEMS = False
    if SKIP_PROBLEMS:
        try:
            metadata, missing, non_uniform = get_metadata_from_catalog_entries(run['col'], target_metadata)
            write_metadata_output(store_name, metadata, missing, non_uniform)
        except Exception as e:
            print(e)
            print(f'Could not produce metadata for store "{store_name}", skipping.\n')
    else:
        metadata, missing, non_uniform = get_metadata_from_catalog_entries(run['col'], target_metadata)
        write_metadata_output(store_name, metadata, missing, non_uniform)


In [ ]:
!date

In [ ]:
paths = run['col'].df['path']
for p in paths:
    print(p.split('/')[-1])

## Alternative to Using the Catalog for Preprocessing:  Load Datasets Directly

In [ ]:
subset_folder = './data-subsets'
fileList = os.listdir(subset_folder)
fileList

In [ ]:
datasets = []
for f in fileList:
    # Create xarray dataset from file.
    filePath = f'{subset_folder}/{f}'
    ds = xr.open_dataset(filePath, use_cftime=True)
    print(filePath)
    print(ds)
    break
    #preprocess(ds)
        
    datasets.append(ds)

In [ ]:
datasets

## Test preprocessing for 360-day calendars

In [ ]:
# Test conditions for 360 calendars
filePath = './data-subsets/subset_tasmax.rcp85.HadGEM2-ES.RegCM4.day.NAM-22i.raw.nc'
ds = xr.open_dataset(filePath, use_cftime=True)
ds

In [ ]:
ds

In [ ]:
ds_processed = preprocess(ds)
ds_processed

### SANDBOX: Code Testing Area

In [ ]:
ds = xr.open_dataset('/Users/bonnland/GitRepos/cesm-lens-zarrification/notebooks/na-cordex/data-subsets/subset_tasmax.rcp85.MPI-ESM-LR.WRF.day.NAM-22i.raw.nc')
ds

In [ ]:
print(ds.data_vars)
print(ds.attrs['title'])

In [ ]:
# Convert dates in the original dataset from the NoLeap to Gregorian calendar
ds['time'] = [convert_to_gregorian(t) for t in ds.time.values]

In [ ]:
# Create a date range on the Gregorian calendar
start_date = ds.time.values[0]
end_date = ds.time.values[-1]

times = xr.DataArray(xr.cftime_range(start=start_date, end=end_date, freq='D', calendar='gregorian'), dims='time')
times

In [ ]:
# Find the leap days in this date range.
is_leap_day = (times.time.dt.month == 2) & (times.time.dt.day == 29)
leap_days = times.where(is_leap_day, drop=True)
leap_days

In [ ]:
# Create fill values for these days.
one_time_step = ds['tasmax'].isel(time=slice(0, 1))
fill_values = []
for leap_day in leap_days:
    d = xr.full_like(one_time_step,fill_value=np.nan)
    d = d.assign_coords(time=[leap_day.data])
    fill_values.append(d)

In [ ]:
# Append the fill values to the dataset and then sort values by time.
fill_values.append(ds['tasmax'])

ds_fixed=xr.concat(fill_values, dim='time').sortby('time')
ds_fixed

In [ ]:
col

In [ ]:
#[dsets[key].get_index('time') for key in dsets][2][0]
[dsets[key].get_index('time') for key in dsets]

In [ ]:
list(dsets.values())[:2]

In [ ]:
list(dsets.values())[0].time.values[0]

In [ ]:
xr.concat(list(dsets.values())[:3], dim='member_id', combine_attrs='drop', data_vars=['tasmax'])

In [ ]:
ds.time.values[0].replace(hour=23)

In [ ]:
preprocess(ds)

In [ ]:
type(ds.time.indexes["time"].to_datetimeindex())


In [ ]:
# Use the following query to gather all data for one variable.
#subset = col.search(variable='tasmax', scenario=['hist','rcp85'], grid='NAM-22i', frequency='day')
subset = col.search(variable='tasmax', scenario=['rcp85'], grid='NAM-22i', frequency='day')

# Use this to load some 360-day data for conversion to the Gregorian calendar.
#subset = col.search(variable='tasmax', scenario=['hist'], grid='NAM-22i', frequency='day', driver='HadGEM2-ES')


subset.unique(columns=['rcm', 'driver', 'biascorrection', 'common'])

In [ ]:
subset.keys()

In [ ]:
subset.df

In [ ]:
for key in subset.keys():
    print(type(subset[key]))

In [ ]:
# Look for strange metadata
for key in tqdm(subset.keys()):
    try:
        subset[key](cdf_kwargs={'chunks': {}, 'decode_times': False}).to_dask()
    except Exception as e:
        print(f'\tFile:{subset[key].df.path.tolist()} --- Exception: {e}', end="")

### Unused machine-dependent Dask invocation: Superseded by using NCARCluster

In [ ]:
import dask

machine = 'cheyenne'  # 'casper'
if machine == 'cheyenne':
    # The following is supposedly set when using NCARCluster
    dask.config.set({'distributed.dashboard.link': "https://jupyterhub.ucar.edu/ch/user/{USER}/proxy/{port}/status"})
    from ncar_jobqueue import NCARCluster
    cluster = NCARCluster(cores=10, processes=20, memory='109GB', project='STDD0003')
    cluster.scale(jobs=20)
else:
    # Assume machine is Casper.
    dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})
    from dask_jobqueue import SLURMCluster
    cluster = SLURMCluster(cores=8, memory='200GB', project='STDD0003')
    cluster.scale(jobs=8)

from distributed import Client
client = Client(cluster)
cluster

#### Metadata Test Code

In [ ]:
m = get_metadata(ds, 'test')
m

In [ ]:
global_metadata_dict = get_metadata_from_catalog_entries(col)
save_metadata_to_csv(global_metadata_dict, "tasmax")